In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

pd.options.display.max_columns = None
pd.options.display.max_rows = None

import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected = True)
from matplotlib import font_manager, rc
plt.style.use('seaborn')
rc('font',family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
final_df = pd.read_csv('ufcdata/final_df.csv',encoding='euc-kr')
final_df.head()

,R_fighter,B_fighter,Winner,date,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_opp_KD,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_opp_PASS,B_avg_opp_REV,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_opp_SIG_STR_pct,B_avg_opp_SUB_ATT,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_total_rounds_fought,B_total_time_fought.seconds.,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO.TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,B_age,R_current_lose_streak,R_current_win_streak,R_avg_BODY_att,R_avg_BODY_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_KD,R_avg_LEG_att,R_avg_LEG_landed,R_avg_PASS,R_avg_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_att,R_avg_TD_landed,R_avg_TD_pct,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_longest_win_streak,R_losses,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_opp_KD,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_opp_PASS,R_avg_opp_REV,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_opp_SIG_STR_pct,R_avg_opp_SUB_ATT,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_opp_TD_pct,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_total_rounds_fought,R_total_time_fought.seconds.,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO.TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age
0,Henry Cejudo,Marlon Moraes,Red,2019-06-08,True,Bantamweight,5,0,4,9.200000,6.000000,0.200000,0.000000,62.600000,20.600000,2.600000,2.000000,48.600000,11.200000,0.800000,7.6,5.400000,0.400000,0.000000,65.40,22.600000,0.466000,0.400000,0.80000,0.200000,0.100000,66.400000,23.600000,4,1,6.400000,4.000000,1.000000,0.60000,51.200000,17.400000,0.600000,0.200000,39.600000,9.400000,0.200000,6.80000,4.800000,0.000000,0.000000,52.800000,18.20000,0.236000,0.000000,1.000000,0.400000,0.100000,53.800000,19.200000,9,419.400000,0,0,1,0,2,1,0,4,Orthodox,167.64,170.18,135,31.0,0,4,21.900000,16.400000,17.000000,11.000000,75.000000,26.500000,9.400000,6.500000,74.200000,23.90,0.400,5.300000,3.700000,1.200000,0.000000,101.400000,44.000000,0.466000,0.100000,5.300000,1.900000,0.458000,129.900000,69.100000,4,2,13.300000,8.800000,7.500000,5.100000,90.500000,26.800000,0.800000,0.300000,76.100000,17.300000,0.100000,9.400000,6.100000,0.000000,0.000000,98.800000,32.200000,0.336000,0.000000,0.900000,0.100000,0.050000,110.500000,43.300000,27,742.60,3,0,2,4,2,0,0,8,Orthodox,162.56,162.56,135,32.0
1,Valentina Shevchenko,Jessica Eye,Red,2019-06-08,True,Women's Flyweight,5,0,3,14.600000,9.100000,11.800000,7.300000,124.700000,42.100000,2.400000,1.900000,112.000000,32.000000,0.000000,12.3,10.200000,0.800000,0.000000,138.90,51.300000,0.399000,0.700000,1.00000,0.500000,0.225000,158.700000,69.600000,3,6,13.000000,9.300000,12.800000,9.60000,101.700000,32.000000,8.100

### UFC 규칙 개정일인 2009-07-30 이전 제거

In [3]:
limit_date = '2009-07-30'
final_df = final_df[(final_df['date'] > limit_date)]
print(final_df.shape)

(3597, 141)


In [4]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3597 entries, 0 to 3596
Columns: 141 entries, R_fighter to R_age
dtypes: bool(1), float64(104), int64(29), object(7)
memory usage: 3.9+ MB


### Label encoding

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [6]:
final_df=final_df.drop(['R_fighter','B_fighter','date'], axis=1)
final_df = final_df[final_df['Winner'] != 'Draw']
list(final_df.select_dtypes(include=['object', 'bool']))

['Winner', 'title_bout', 'weight_class', 'B_Stance', 'R_Stance']

In [7]:
final_df.Winner.unique()

array(['Red', 'Blue'], dtype=object)

In [8]:
final_df['Winner'] = le.fit_transform(final_df['Winner'])
final_df['title_bout'] = le.fit_transform(final_df['title_bout'])

### Dummy

In [9]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data

In [10]:
dummy_columns = ['weight_class','B_Stance','R_Stance']
final_df=dummy_data(final_df,dummy_columns)

In [11]:
list(final_df.select_dtypes(include=['object', 'bool']))

[]

In [12]:
final_df.loc[final_df['R_age']<=32, 'R_age'] =0
final_df.loc[final_df['R_age']>32, 'R_age'] = 1
final_df['R_age'] = final_df['R_age'].map( { 0.0: 'young',  1.0: 'nice_age'} ).astype(str)

In [13]:
final_df.loc[final_df['B_age']<=32, 'B_age'] =0
final_df.loc[final_df['B_age']>32, 'B_age'] = 1
final_df['B_age'] = final_df['B_age'].map( { 0.0: 'young',  1.0: 'nice_age'} ).astype(str)

In [14]:
final_df['R_age'] = le.fit_transform(final_df['R_age'])
final_df['B_age'] = le.fit_transform(final_df['B_age'])

In [15]:
dummy_columns = ['R_age','B_age']
final_df=dummy_data(final_df,dummy_columns)

In [16]:
final_df.head()

,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_opp_KD,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_opp_PASS,B_avg_opp_REV,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_opp_SIG_STR_pct,B_avg_opp_SUB_ATT,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_total_rounds_fought,B_total_time_fought.seconds.,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO.TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_avg_BODY_att,R_avg_BODY_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_KD,R_avg_LEG_att,R_avg_LEG_landed,R_avg_PASS,R_avg_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_att,R_avg_TD_landed,R_avg_TD_pct,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_longest_win_streak,R_losses,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_opp_KD,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_opp_PASS,R_avg_opp_REV,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_opp_SIG_STR_pct,R_avg_opp_SUB_ATT,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_opp_TD_pct,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_total_rounds_fought,R_total_time_fought.seconds.,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO.TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Welterweight,weight_class_Women's Bantamweight,weight_class_Women's Featherweight,weight_class_Women's Flyweight,weight_class_Women's Strawweight,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch,R_age_0,R_age_1,B_age_0,B_age_1
0,1,1,5,0,4,9.200000,6.000000,0.200000,0.000000,62.600000,20.600000,2.600000,2.000000,48.600000,11.200000,0.800000,7.6,5.400000,0.400000,0.000000,65.40,22.600000,0.466000,0.400000,0.80000,0.200000,0.100000,66.400000,23.600000,4,1,6.400000,4.000000,1.000000,0.60000,51.200000,17.400000,0.600000,0.200000,39.600000,9.400000,0.200000,6.80000,4.800000,0.000000,0.000000,52.800000,18.20000,0.236000,0.000000,1.000000,0.400000,0.100000,53.800000,19.200000,9,419.400000,0,0,1,0,2,1,0,4,167.64,170.18,135,0,4,21.900000,16.400000,17.000000,11.000000,75.000000,26.500000,9.400000,6.500000,74.200000,23.90,0.400,5.300000,3.700000,1.200000,0.000000,101.400000,44.000000,0.466000,0.100000,5.300000,1.900000,0.458000,129.900000,69.100000,4,2,13.300000,8.800000,7.500000,5.100000,90.500000,26.800000,0.800000,0.300000,76.100000,17.300000,0.100000,9.400000,6.100000,0.000000,0.000000,98.800000,32.200000,0.336000,0.000000,0.900000,0.100000,0.050000,110.500000,43.300000,27,742.60,3,0,2,4,

### Scailing

In [17]:
from sklearn.preprocessing import StandardScaler
data_num = final_df.select_dtypes(include=['float64','int64'])

scaler = StandardScaler()
final_df[list(data_num.columns)] = scaler.fit_transform(final_df[list(data_num.columns)])

### Train - Test split

In [18]:
from sklearn.model_selection import train_test_split
y = final_df.Winner.values
X = final_df.drop(columns = 'Winner').values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)

### Modeling - MLP Classifier

In [19]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X_train,y_train)

print("훈련 세트 정확도: {:.3f}".format(mlp.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(mlp.score(X_test, y_test)))

훈련 세트 정확도: 1.000
테스트 세트 정확도: 0.583


D:\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



In [20]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation= 'tanh',alpha=0.1,
                    early_stopping=True,
                    validation_fraction=0.1,
                    random_state=23,
                    hidden_layer_sizes=(150,))
mlp.fit(X_train,y_train)

MLPClassifier(activation='tanh', alpha=0.1, early_stopping=True,
              hidden_layer_sizes=(150,), random_state=23)

In [21]:
print("훈련 세트 정확도: {:.3f}".format(mlp.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(mlp.score(X_test, y_test)))

훈련 세트 정확도: 0.719
테스트 세트 정확도: 0.649


**교차검증**

In [22]:
from sklearn.model_selection import cross_validate
res = cross_validate(mlp,X,y, cv=10, scoring="accuracy", return_train_score = True )

res_df = pd.DataFrame(res)
display(res_df)
print("평균 시간과 점수:\n",res_df.mean())

,fit_time,score_time,test_score,train_score
0,0.602775,0.000000,0.584746,0.720528
1,0.572334,0.000000,0.632768,0.705439
2,0.608301,0.000000,0.615819,0.695379
3,0.804218,0.000000,0.635593,0.746935
4,0.834860,0.000000,0.610169,0.748821
5,1.472975,0.000000,0.620397,0.872093
6,0.800187,0.001995,0.662890,0.740415
7,1.012240,0.001995,0.623229,0.791326
8,0.811893,0.000000,0.651558,0.741043
9,0.557723,0.002988,0.640227,0.690761


평균 시간과 점수:
 fit_time       0.807751
score_time     0.000698
test_score     0.627740
train_score    0.745274
dtype: float64


### Modeling - GradientBoosting Classifier

In [23]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier()
gbrt.fit(X_train,y_train)

print("훈련 세트 점수: {:.3f}".format(gbrt.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(gbrt.score(X_test, y_test)))

훈련 세트 점수: 0.819
테스트 세트 점수: 0.619


In [24]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(random_state = 123,
                                  learning_rate = 0.1,
                                  min_samples_leaf = 200,
                                 )
gbrt.fit(X_train,y_train)

GradientBoostingClassifier(min_samples_leaf=200, random_state=123)

In [25]:
print("훈련 세트 점수: {:.3f}".format(gbrt.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(gbrt.score(X_test, y_test)))

훈련 세트 점수: 0.767
테스트 세트 점수: 0.629


**교차검증**

In [26]:
from sklearn.model_selection import cross_validate
res = cross_validate(gbrt,X,y, cv=5, scoring="accuracy", return_train_score = True )

res_df = pd.DataFrame(res)
display(res_df)
print("평균 시간과 점수:\n",res_df.mean())

,fit_time,score_time,test_score,train_score
0,4.919071,0.000000,0.625177,0.769802
1,5.000503,0.002991,0.635078,0.756365
2,5.150995,0.002992,0.594059,0.766620
3,6.104126,0.000000,0.629420,0.747171
4,5.642631,0.001994,0.642150,0.761315


평균 시간과 점수:
 fit_time       5.363465
score_time     0.001595
test_score     0.625177
train_score    0.760255
dtype: float64


### Modeling - AdaBoost Classifier

In [27]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(X_train,y_train)

print("훈련 세트 정확도: {:.3f}".format(ada.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(ada.score(X_test, y_test)))

훈련 세트 정확도: 0.700
테스트 세트 정확도: 0.622


In [28]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(random_state=1234, learning_rate=0.4)
ada.fit(X_train,y_train)

AdaBoostClassifier(learning_rate=0.4, random_state=1234)

In [29]:
print("훈련 세트 정확도: {:.3f}".format(ada.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(ada.score(X_test, y_test)))

훈련 세트 정확도: 0.684
테스트 세트 정확도: 0.636


**교차검증**

In [30]:
from sklearn.model_selection import cross_validate
res = cross_validate(ada,X,y, cv=5, scoring="accuracy", return_train_score = True )

res_df = pd.DataFrame(res)
display(res_df)
print("평균 시간과 점수:\n",res_df.mean())

,fit_time,score_time,test_score,train_score
0,1.220824,0.015622,0.636492,0.682461
1,1.249801,0.031242,0.650636,0.681400
2,1.187296,0.015606,0.584158,0.692008
3,1.159832,0.015573,0.652051,0.681047
4,1.285271,0.015625,0.650636,0.676803


평균 시간과 점수:
 fit_time       1.220605
score_time     0.018734
test_score     0.634795
train_score    0.682744
dtype: float64
